In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wave
import gensim
import soundfile as sf
import torch
from scipy.io import wavfile
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [2]:
df = pd.read_csv('./playlist_data.csv')

In [3]:
df

,playlist_id,playlist,genre
0,37i9dQZF1DXcBWIGoYBM5M,"['Heat Waves Glass Animals', 'Ghost Justin Bie...",pop
1,37i9dQZF1DX0kbJZpiYdZl,"['Big Energy Latto', 'Ghost Justin Bieber', 'H...",pop
2,37i9dQZF1DX4JAvHpjipBk,"['CHICKEN TERIYAKI ROSALÍA', 'Usain Boo Kodak ...",pop
3,37i9dQZF1DWUa8ZRTfalHk,"['Go Cat Burns', ""she's all i wanna be Tate Mc...",pop
4,37i9dQZF1DX5Vy6DFOcx00,"['Mount Everest Labrinth', 'MIDDLE OF THE NIGH...",pop
...,...,...,...
195,37i9dQZF1DX6RbMTl2YnQ8,"['Vía Láctea Zoé', 'Irresponsables Babasónicos...",rock
196,37i9dQZF1DXbFRZSqP41al,"['Hash Pipe Weezer', 'Celebrity Skin Hole', 'A...",rock
197,37i9dQZF1DX9E1mLvTvD1f,['Bullet With Butterfly Wings - Remastered 201...,rock
198,37i9dQZF1DX8V6C5Nr1MvC,['Good Riddance (Time of Your Life) Green Day'...,rock


In [18]:
playlist = df.playlist.apply(gensim.utils.simple_preprocess)
playlist

0      [heat, waves, glass, animals, ghost, justin, b...
1      [big, energy, latto, ghost, justin, bieber, he...
2      [chicken, teriyaki, rosalía, usain, boo, kodak...
3      [go, cat, burns, she, all, wanna, be, tate, mc...
4      [mount, everest, labrinth, middle, of, the, ni...
                             ...                        
195    [vía, láctea, zoé, irresponsables, babasónicos...
196    [hash, pipe, weezer, celebrity, skin, hole, ar...
197    [bullet, with, butterfly, wings, remastered, t...
198    [good, riddance, time, of, your, life, green, ...
199    [smile, wolf, alice, its, alright, kevin, walk...
Name: playlist, Length: 200, dtype: object

In [19]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [21]:
model.build_vocab(playlist, progress_per=1000)

In [22]:
model.epochs

5

In [23]:
model.corpus_count

200

In [24]:
model.train(playlist, total_examples=model.corpus_count, epochs=model.epochs)

(343330, 402115)

In [30]:
model.wv[playlist[0]]

array([[-0.3943806 ,  0.12394643,  0.07046469, ..., -0.31165636,
        -0.2348632 , -0.14245756],
       [-0.50555813,  0.16056477,  0.08717586, ..., -0.38697568,
        -0.30969104, -0.18127958],
       [-0.6876205 ,  0.22002119,  0.10509502, ..., -0.5316136 ,
        -0.40480646, -0.266519  ],
       ...,
       [-0.10574312,  0.03102683,  0.01986708, ..., -0.09099478,
        -0.06987097, -0.03513696],
       [-0.25233012,  0.0941081 ,  0.03385782, ..., -0.19368301,
        -0.16607212, -0.10819558],
       [-0.20890875,  0.06279965,  0.03464428, ..., -0.14860454,
        -0.11691155, -0.06463688]], dtype=float32)

In [32]:
model.wv[playlist[61]]

array([[-0.40036762,  0.13136783,  0.07245627, ..., -0.32305455,
        -0.24876761, -0.14951962],
       [-0.1313086 ,  0.04469064,  0.01200642, ..., -0.09438103,
        -0.07211262, -0.05008363],
       [-0.15024027,  0.04305934,  0.01933489, ..., -0.10994247,
        -0.08216745, -0.05321605],
       ...,
       [-0.17122744,  0.05289279,  0.02968975, ..., -0.12570916,
        -0.10801583, -0.06864376],
       [-1.1463795 ,  0.39179462,  0.16560316, ..., -0.94404566,
        -0.7410337 , -0.46997336],
       [-0.2014305 ,  0.06789634,  0.04019691, ..., -0.15208685,
        -0.12562342, -0.08002275]], dtype=float32)